# Análisis del riesgo de incumplimiento de los prestatarios

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Averiguaremosr si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Se creará una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

La finalidad del proyecto es verificar si hay una relación entre el pago a tiempo de un préstamo y la cantidad de hijos, estado civil, el nivel de ingresos o el propósito del préstamo.

 **Indice del contenido**

1. Carga de información general
2. Exploración de datos
3. Transformación de datos
    3.1 Restauración de valores erróneos en 'children'
    3.2 Restauración de valores ausentes en 'total_income'
    3.3 Restauración de valores en 'days_employed'
4. Clasificación de datos
5. Hipótesis
6. Conclusión general

## Abre el archivo de datos y mira la información general. 


In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np

In [2]:
# Carga los datos
try:
    dataset=pd.read_csv('/credit_scoring_eng.csv')
except:
    dataset=pd.read_csv('/datasets/credit_scoring_eng.csv')
    
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo


In [3]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
filas = len(dataset.axes[0]) 
columnas = len(dataset.axes[1]) 
  
print('Número de filas: ' + str(filas)) 
print('Número de columnas: ' + str(columnas)) 

Número de filas: 21525
Número de columnas: 12


In [4]:
# vamos a mostrar las primeras filas
dataset.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


En la tabla mostrada, se aprecian valores negativos en la columna 'days_employed', valores repetidos con diferencias ortográficas en la columna 'education' y valores que se pueden categorizar en las columnas 'dob_years' y 'purpose'. Es necesario revisar si hay valores ausentes y correlación entre valores ausentes de distintas columnas.

In [5]:
# Obtenemos información sobre los datos
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Hay valores ausentes en las columnas de days_employed y de total_income, posiblemente en las mismas filas.

In [6]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
dataset[dataset['days_employed'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Los valores ausentes en las columnas 'days_employed' y 'total_income' parecen simétricos. Esto podría deberse a que el cálculo para el ingreso total depende de la variables de 'days_employed'. 

In [7]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
lista_nulos=dataset[dataset['days_employed'].isna()]
lista_nulos

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusión intermedia**

El número de filas en la tabla filtrada, que muestra las filas de valores ausentes en las columnas days_employed y total_income,coincide con el número de valores ausentes de la columna days_employed, por lo que se puede afirmar que hay una correlación entre ambas variables.

In [8]:
#Porcentaje de valores ausentes
dataset['days_employed'].isna().sum()*100/len(dataset['days_employed'])

10.099883855981417

El porcentaje de los valores ausentes en comparación con el conjuto de datos completos es de 10.09 %, por lo que sería necesario completar tales valores. Corroboraremos si los valores ausentes están relacionados al tipo de educación y si dependen de otra variable del dataframe. 

[Explica tus próximos pasos y cómo se correlacionan con las conclusiones que has hecho hasta ahora.]

In [9]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
lista_nulos[['days_employed','education']]['education'].value_counts()

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
SOME COLLEGE              7
Some College              7
PRIMARY EDUCATION         1
Primary Education         1
Name: education, dtype: int64

In [10]:
# Comprobación de la distribución
lista_nulos.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


No existe una relación entre las columnas 'days_employed' y 'total_income' con otras variables; sin embargo, sí existe una correlación entre estas dos.

**Posibles razones por las que hay valores ausentes en los datos**

Es posible que los valores ausentes de 'total_income' puedan deberse a que el cálculo de dicha variable dependa de la cantidad de días empleados.

In [11]:
# Comprobando la distribución en el conjunto de datos entero
dataset.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Conclusión intermedia**

La distribución en el conjunto de datos origial es similar a la tabla filtrada, por lo que se podría usar para rellenar los valores ausentes.

In [12]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes
lista_nulos.groupby('gender')['education'].value_counts()


gender  education          
F       secondary education    962
        bachelor's degree      338
        SECONDARY EDUCATION     46
        Secondary Education     45
        some college            34
        Bachelor's Degree       20
        BACHELOR'S DEGREE       15
        primary education       13
        Some College             5
        SOME COLLEGE             4
        PRIMARY EDUCATION        1
        Primary Education        1
M       secondary education    446
        bachelor's degree      158
        SECONDARY EDUCATION     21
        some college            21
        Secondary Education     20
        BACHELOR'S DEGREE        8
        primary education        6
        Bachelor's Degree        5
        SOME COLLEGE             3
        Some College             2
Name: education, dtype: int64

**Conclusión intermedia**

No hay una correlación directa entre los valores ausentes y las variables de género ni el nivel de educación.

In [13]:
# Comprobación de otros patrones: explica cuáles
lista_nulos.groupby('children')['education'].value_counts()

children  education          
-1        secondary education      2
          Secondary Education      1
 0        secondary education    953
          bachelor's degree      307
          SECONDARY EDUCATION     50
          Secondary Education     45
          some college            33
          primary education       14
          BACHELOR'S DEGREE       13
          Bachelor's Degree       12
          Some College             6
          SOME COLLEGE             5
          Primary Education        1
 1        secondary education    291
          bachelor's degree      126
          some college            14
          Secondary Education     12
          Bachelor's Degree        9
          SECONDARY EDUCATION      9
          BACHELOR'S DEGREE        7
          primary education        4
          SOME COLLEGE             2
          Some College             1
 2        secondary education    128
          bachelor's degree       50
          SECONDARY EDUCATION      8
        

In [14]:
dataset.groupby('children')['education'].value_counts()

children  education          
-1        secondary education    31
          bachelor's degree       8
          SECONDARY EDUCATION     4
          Secondary Education     2
          BACHELOR'S DEGREE       1
                                 ..
 20       bachelor's degree      12
          Secondary Education     6
          BACHELOR'S DEGREE       2
          SECONDARY EDUCATION     2
          some college            2
Name: education, Length: 68, dtype: int64

**Conclusiones**

No se encontraron patrones que relacionen los valores ausentes con alguna otra variable.

Los valores ausentes serán reemplazados por la media o mediana de las variables más significativas encontradas. Se realizarán las correcciones ortográficas y eliminación de duplicados encontrados.

## Transformación de datos

Realizaremos la eliminación de duplicados y correcciones ortográficas en la columna 'education', corrección de valores incongruentes en las columnas 'children' y 'days_employed'. 

In [15]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
dataset['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [16]:
# Arreglamos los registros si es necesario
dataset['education']=dataset['education'].str.lower()

In [17]:
# Comprobamos todos los valores en la columna para asegurarnos de que los hayamos corregido
dataset['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

### Restaurar valores erróneos en Children

In [18]:
# Veamos la distribución de los valores en la columna `children`
dataset['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [19]:
#porcentaje de datos problemáticos
(len(dataset[dataset['children']==-1]) + len(dataset[dataset['children']==-1]))*100/len(dataset['children'])

0.4367015098722416

Existen 94 datos problemáticos que representan el 0.43 % del total de los datos en la columna 'children', por lo que serán reemplazados.

In [20]:
# Eliminamos los datos con cantidad de hijos de negativos, en lugar de asumir errores.
dataset['children']=dataset['children'].replace(-1,1)
dataset['children']=dataset['children'].replace(20,2)

In [21]:
# Comprobamos la columna `children` de nuevo para asegurarnos de que todo está arreglado
dataset['children'].unique()


array([1, 0, 3, 2, 4, 5])

En la columna `days_employed` hay números negativos también; sin embargo, esto puede deberse a la forma en la que se obtuvo el registro de estos valores. De esta forma, los valores negativos serán convertidos a positivos si el porcentaje es muy alto.

In [22]:
len(dataset[dataset['days_employed']<0])*100/len(dataset['days_employed'])

73.89547038327527

Debido a que la cantidad de datos problemáticos es alta, esto podría deberse a problemas técnicos. Se aplicará el valor absoluto a dichos valores en lugar de ser eliminados.

In [23]:
# Abordamos los valores problemáticos, si existen.
dataset['days_employed']=abs(dataset['days_employed'])

In [24]:
dataset['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [25]:
dataset['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Las filas en las que 'dob_years' sea igual a 0 fueron reemplazadas por el valor promedio de las edades.

In [26]:
dataset['dob_years']=dataset['dob_years'].replace(0,dataset['dob_years'].mean())

In [27]:
dataset['dob_years'].describe()

count    21525.000000
mean        43.496522
std         12.218174
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [28]:
# Veamos los valores de la columna
dataset['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [29]:
# Aborda los valores problemáticos en `family_status`, si existen
dataset['family_status']=dataset['family_status'].replace('divorced','single')
dataset['family_status']=dataset['family_status'].replace('unmarried','single')

In [30]:
# Comprobamos
dataset['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'single'],
      dtype=object)

Revisamos la columna `gender`.

In [31]:
dataset['gender'].value_counts()# Veamos los valores en la columna

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Revisamos la columna `income_type`.

In [32]:
# Veamos los valores en la columna
dataset['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [33]:
# Aborda los valores problemáticos, si existen
dataset['income_type']=dataset['income_type'].replace('entrepreneur','business')

In [34]:
# Comprobamos
dataset['income_type'].value_counts()

employee                       11119
business                        5087
retiree                         3856
civil servant                   1459
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [35]:
# Comprobamos los duplicados
dataset[dataset.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41.0,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34.0,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64.0,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60.0,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47.0,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30.0,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [36]:
dataset=dataset.drop_duplicates().reset_index(drop=True)# Aborda los duplicados, si existen

In [37]:
# Última comprobación para ver si tenemos duplicados
dataset[dataset.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [38]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
filas = len(dataset.axes[0]) 
columnas = len(dataset.axes[1]) 
  
print('Número de filas: ' + str(filas)) 
print('Número de columnas: ' + str(columnas)) 

Número de filas: 21454
Número de columnas: 12


In [93]:
(21525-21454)*100/21525 

0.32984901277584205

Se redujo el 0.32% de filas de datos por tener información duplicada.


# Trabajar con valores ausentes

Se usarán los diccionarios de family_status_id y family_status para simplificar el código y evitar errores ortográficos. Se evitará usar diccionarios para 'education' pues es necesario realizar ciertas correcciones primero.

In [40]:
# Encuentra los diccionarios
dicc_family_stat = {0:'married',1:'civil partnership',2:'widow / widower',3:'single',4:'single'}

### Restauramos valores ausentes en `total_income`

La columna 'total_income' contiene valores ausentes que serán reemplazados por el valor de la media o mediana del 'total_income' en función de los parámetros que tengan la distribución que mejor se adecúe al caso. 

In [41]:
# Vamos a escribir una función que calcule la categoría de edad
def categ_edad(edad):
    if edad<=18:
        return '0 - 18'
    elif edad<30:
        return '19 - 29'
    elif edad<40:
        return '30 - 39'
    elif edad<50:
        return '40 - 49'
    elif edad<60:
        return '50 - 59'
    elif edad<70:
        return '60 - 69'
    else:
        return 'mayor a 70'

In [42]:
# Probamos si la función funciona bien
categ_edad(45)

'40 - 49'

In [43]:
# Creamos una nueva columna basada en la función
dataset['categ_edad']=dataset['dob_years'].apply(categ_edad)

In [44]:
# Comprobamos cómo los valores en la nueva columna
dataset['categ_edad']

0        40 - 49
1        30 - 39
2        30 - 39
3        30 - 39
4        50 - 59
          ...   
21449    40 - 49
21450    60 - 69
21451    30 - 39
21452    30 - 39
21453    40 - 49
Name: categ_edad, Length: 21454, dtype: object

In [45]:
# Creamos una tabla sin valores ausentes y mostramos algunas de sus filas para asegurarte de que se ve bien
dataset_no_null=dataset.dropna()
dataset_no_null.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,categ_edad
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40 - 49
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30 - 39
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30 - 39
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30 - 39
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50 - 59
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,19 - 29
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40 - 49
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,50 - 59
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30 - 39
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40 - 49


In [46]:
# Examinamos los valores medios de los ingresos en función de los factores que identificaste
dataset_no_null.groupby('categ_edad')['total_income'].mean().round(2)

categ_edad
19 - 29       25533.96
30 - 39       28312.48
40 - 49       28491.93
50 - 59       25811.70
60 - 69       23242.81
mayor a 70    20125.66
Name: total_income, dtype: float64

In [47]:
# Examinamos los valores medianos de los ingresos en función de los factores que identificaste
dataset_no_null.groupby('categ_edad')['total_income'].median().round(2)

categ_edad
19 - 29       22742.65
30 - 39       24667.53
40 - 49       24755.70
50 - 59       22203.07
60 - 69       19817.44
mayor a 70    18751.32
Name: total_income, dtype: float64

In [48]:
dataset_no_null.groupby('education')['total_income'].mean().round(2)

education
bachelor's degree      33142.80
graduate degree        27960.02
primary education      21144.88
secondary education    24594.50
some college           29045.44
Name: total_income, dtype: float64

In [49]:
dataset_no_null.groupby('education')['total_income'].median().round(2)

education
bachelor's degree      28054.53
graduate degree        25161.58
primary education      18741.98
secondary education    21836.58
some college           25618.46
Name: total_income, dtype: float64

In [50]:
dataset_no_null.groupby('gender')['total_income'].mean().round(2)

gender
F      24655.60
M      30907.14
XNA    32624.82
Name: total_income, dtype: float64

In [51]:
dataset_no_null.groupby('gender')['total_income'].median().round(2)

gender
F      21464.84
M      26834.30
XNA    32624.82
Name: total_income, dtype: float64

In [52]:
dataset_no_null.groupby('income_type')['total_income'].mean().round(2)

income_type
business                       32397.17
civil servant                  27343.73
employee                       25820.84
paternity / maternity leave     8612.66
retiree                        21940.39
student                        15712.26
unemployed                     21014.36
Name: total_income, dtype: float64

In [53]:
dataset_no_null.groupby('income_type')['total_income'].median().round(2)

income_type
business                       27583.36
civil servant                  24071.67
employee                       22815.10
paternity / maternity leave     8612.66
retiree                        18962.32
student                        15712.26
unemployed                     21014.36
Name: total_income, dtype: float64

Las características que mejor definen los ingresos debido a la mayor variación entre mínimos y máximos sería la mediana de la columna 'income_type'.

In [54]:
median_total_income=dataset_no_null.groupby('income_type')['total_income'].median().round(2)
median_total_income

income_type
business                       27583.36
civil servant                  24071.67
employee                       22815.10
paternity / maternity leave     8612.66
retiree                        18962.32
student                        15712.26
unemployed                     21014.36
Name: total_income, dtype: float64

In [55]:
dataset_no_null.groupby('income_type')['total_income'].median().round(2).loc['employee']

22815.1

In [56]:
#  Escribimos una función que usaremos para completar los valores ausentes
def reemplazar_ausentes(df):
    df1=dataset.loc[dataset['income_type']=='business','days_employed'].fillna(value=median_total_income.loc['business']) #.isna().sum()#=123 #
    df1=df1.append(dataset.loc[dataset['income_type']=='employee','days_employed'].fillna(value=median_total_income.loc['employee'])).sort_index()
    df1=df1.append(dataset.loc[dataset['income_type']=='paternity / maternity leave','days_employed'].fillna(value=median_total_income.loc['paternity / maternity leave'])).sort_index()
    df1=df1.append(dataset.loc[dataset['income_type']=='retiree','days_employed'].fillna(value=median_total_income.loc['retiree'])).sort_index()
    df1=df1.append(dataset.loc[dataset['income_type']=='student','days_employed'].fillna(value=median_total_income.loc['student'])).sort_index()
    df1=df1.append(dataset.loc[dataset['income_type']=='unemployed','days_employed'].fillna(value=median_total_income.loc['unemployed'])).sort_index()
    df1=df1.append(dataset.loc[dataset['income_type']=='civil servant','days_employed'].fillna(value=median_total_income.loc['civil servant'])).sort_index()
    return df1

In [57]:
# Comprobamos si funciona
dataset['total_income'].isna().sum()

2103

In [58]:
dataset['total_income']=reemplazar_ausentes(dataset)

In [59]:
dataset['total_income'].isna().sum()

0

In [60]:
# Reemplazamos los valores ausentes si hay algún error
dataset['total_income'].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
21449    False
21450    False
21451    False
21452    False
21453    False
Name: total_income, Length: 21454, dtype: bool

In [61]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  categ_edad        21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


###  Restauración de valores en `days_employed`

La columna 'days_employed' contiene valores ausentes que serán reemplazados por el valor de la media o mediana del 'days_employed' en función de los parámetros que tengan la distribución que mejor se adecúe al caso. Se hará las comparaciones para las columnas 'categ_edad', 'education' e 'income_type'.

In [62]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
dataset_no_null.groupby('categ_edad')['days_employed'].median().round(2)

categ_edad
19 - 29          999.03
30 - 39         1601.78
40 - 49         2108.53
50 - 59         4796.77
60 - 69       354935.62
mayor a 70    361336.99
Name: days_employed, dtype: float64

In [63]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
dataset_no_null.groupby('categ_edad')['days_employed'].mean().round(2)

categ_edad
19 - 29         2082.49
30 - 39         4155.03
40 - 49        13439.23
50 - 59       132907.55
60 - 69       283926.48
mayor a 70    320819.15
Name: days_employed, dtype: float64

In [64]:
dataset_no_null.groupby('education')['days_employed'].median().round(2)

education
bachelor's degree      1895.75
graduate degree        5660.06
primary education      3043.93
secondary education    2392.48
some college           1209.13
Name: days_employed, dtype: float64

In [65]:
dataset_no_null.groupby('education')['days_employed'].mean().round(2)

education
bachelor's degree       42375.41
graduate degree        121323.63
primary education      130340.43
secondary education     76413.82
some college            20656.63
Name: days_employed, dtype: float64

In [66]:
dataset_no_null.groupby('income_type')['days_employed'].median().round(2)

income_type
business                         1546.33
civil servant                    2689.37
employee                         1574.20
paternity / maternity leave      3296.76
retiree                        365213.31
student                           578.75
unemployed                     366413.65
Name: days_employed, dtype: float64

In [67]:
dataset_no_null.groupby('income_type')['days_employed'].mean().round(2)

income_type
business                         2111.18
civil servant                    3399.90
employee                         2326.50
paternity / maternity leave      3296.76
retiree                        365003.49
student                           578.75
unemployed                     366413.65
Name: days_employed, dtype: float64

Se usarán los promedios de 'income_type' para reemplazar los valores ausentes en 'days_employed'. Si bien no hay mucha diferencia comparando con las medianas, los promedios de business y employee tienen un valor más cercano al promedio y mediana general de 'income_type'.

In [68]:
# Escribimos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
def calcular_medias(parametro):
    media=dataset_no_null.groupby(parametro)['days_employed'].mean().round(2)
    return media

In [69]:
# Comprobamos que la función funciona
calcular_medias(dataset['gender'])

gender
F      82397.77
M      37000.33
XNA     2358.60
Name: days_employed, dtype: float64

In [70]:
# Aplicamos la función al income_type
medias_income_type=calcular_medias(dataset['income_type'])

In [71]:
# Comprobamos si la función funcionó
medias_income_type

income_type
business                         2111.18
civil servant                    3399.90
employee                         2326.50
paternity / maternity leave      3296.76
retiree                        365003.49
student                           578.75
unemployed                     366413.65
Name: days_employed, dtype: float64

In [73]:
dataset['days_employed'].isna().sum()

2103

In [74]:
medias_income_type.loc['retiree']

365003.49

In [75]:
# Reemplazar valores ausentes
df2=dataset.loc[dataset['income_type']=='business','days_employed'].fillna(value=medias_income_type.loc['business']) #.isna().sum()#=123 #
df2=df2.append(dataset.loc[dataset['income_type']=='employee','days_employed'].fillna(value=medias_income_type.loc['employee'])).sort_index()
df2=df2.append(dataset.loc[dataset['income_type']=='paternity / maternity leave','days_employed'].fillna(value=medias_income_type.loc['paternity / maternity leave'])).sort_index()
df2=df2.append(dataset.loc[dataset['income_type']=='retiree','days_employed'].fillna(value=medias_income_type.loc['retiree'])).sort_index()
df2=df2.append(dataset.loc[dataset['income_type']=='student','days_employed'].fillna(value=medias_income_type.loc['student'])).sort_index()
df2=df2.append(dataset.loc[dataset['income_type']=='unemployed','days_employed'].fillna(value=medias_income_type.loc['unemployed'])).sort_index()
df2=df2.append(dataset.loc[dataset['income_type']=='civil servant','days_employed'].fillna(value=medias_income_type.loc['civil servant'])).sort_index()

In [76]:
dataset['days_employed']=df2

In [77]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  categ_edad        21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


Habiendo realizado el reemplazo de los valores ausentes, todas las columnas poseen la misma cantidad de filas.

## Clasificación de datos

Se clasificaron los datos de 'purpose' resumidos en categorías de tal forma que se eviten duplicados no implícitos.

In [78]:
# Mostramos los valores de los datos seleccionados para la clasificación
dataset['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            768
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [79]:
# Comprobamos los valores únicos
dataset['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Se identificaron 5 grupos principales: car, wedding, education, housing y business. 

In [80]:
# Escribimos una función para clasificar los datos en función de temas comunes
def categ_purp(purpose):
    if purpose=='supplementary education' or purpose=='education' or purpose=='to become educated' or purpose=='going to university'or purpose=='getting an education' or purpose=='university education' or purpose =='profile education' or purpose=='getting higher education'or purpose=='to get a supplementary education':
        return 'education'
    elif purpose=='purchase of the house' or purpose=='construction of own property' or purpose=='buy residential real estate' or purpose=='building a property' or purpose=='property' or purpose=='housing' or purpose=='building a real estate' or purpose=='housing transactions' or purpose=='purchase of the house for my family' or purpose=='housing renovation' or purpose=='purchase of my own house' or purpose=='buy real estate':
        return 'housing'
    elif purpose=='cars' or purpose=='car purchase' or purpose =='purchase of a car' or purpose=='to buy a car' or purpose=='to own a car' or purpose=='second-hand car purchase'or purpose=='buying a second-hand car'or purpose=='buying my own car':
        return 'car'
    elif purpose=='to have a wedding' or purpose=='wedding ceremony'or purpose=='having a wedding':
        return 'wedding'
    elif purpose=='buying property for renting out' or purpose=='transactions with commercial real estate' or purpose=='transactions with my real estate'or purpose=='buy commercial real estate' or purpose=='real estate transactions':
        return 'business'

In [81]:
#función que crea tabla 
def clasificacion(category):
    data_pivot = dataset.pivot_table(index=category, columns='debt', aggfunc='count')
    return data_pivot

In [82]:
# Creamos una columna con las categorías y cuenta los valores en ellas
dataset['categ_purpose']=dataset['purpose'].apply(categ_purp)
dataset['categ_purpose'].value_counts()

housing      7547
education    4013
car          3812
business     3264
wedding      2324
Name: categ_purpose, dtype: int64

In [83]:
# Revisamos todos los datos numéricos en la columna seleccionada para la clasificación
clasificacion('categ_purpose')['categ_edad']

debt,0,1
categ_purpose,,
business,3008,256
car,3451,361
education,3643,370
housing,7021,526
wedding,2138,186


In [84]:
dataset['categ_purpose'].describe()

count       20960
unique          5
top       housing
freq         7547
Name: categ_purpose, dtype: object

Se realizó una clasificación de los datos numéricos en 'total_income' dividido en cuartiles.

In [85]:
# Creamos una función para clasificar en diferentes grupos numéricos basándose en rangos

#cuartiles de 'total_income'
q=dataset['total_income'].quantile([.25, .5, .75])

#función que clasificará según cuartiles de 'total_income' 
def categ_rang(column):
    if column<=q[0.25]:
        return '1er cuartil'
    elif column<=q[0.5]:
        return '2do cuartil'
    elif column<=q[0.75]:
        return '3er cuartil'
    else:
        return '4to cuartil'    

In [86]:
dataset['cuartil_total_income']=dataset['total_income'].apply(categ_rang)

In [87]:
dataset['cuartil_total_income'].value_counts()

3er cuartil    5565
1er cuartil    5364
2do cuartil    5363
4to cuartil    5162
Name: cuartil_total_income, dtype: int64

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [88]:
tabla_1=clasificacion('children')['categ_edad']
tabla_1['total']=tabla_1[0]+tabla_1[1]
tabla_1['tasa']=tabla_1[1]/tabla_1['total']
tabla_1


debt,0,1,total,tasa
children,,,,
0,13028.0,1063.0,14091.0,0.075438
1,4410.0,445.0,4855.0,0.091658
2,1926.0,202.0,2128.0,0.094925
3,303.0,27.0,330.0,0.081818
4,37.0,4.0,41.0,0.097561
5,9.0,NaN,NaN,NaN


In [89]:
tabla_1_2=dataset.pivot_table(index=('children','categ_edad'), columns='debt', aggfunc='count')['categ_purpose']
tabla_1_2['total']=tabla_1_2[0]+tabla_1_2[1]
tabla_1_2['tasa']=tabla_1_2[1]/tabla_1_2['total']
tabla_1_2

debt                      0      1   total      tasa
children categ_edad                                 
0        19 - 29     1669.0  204.0  1873.0  0.108916
         30 - 39     2061.0  217.0  2278.0  0.095259
         40 - 49     3130.0  248.0  3378.0  0.073416
         50 - 59     3703.0  252.0  3955.0  0.063717
         60 - 69     2006.0  108.0  2114.0  0.051088
         mayor a 70   151.0    4.0   155.0  0.025806
1        19 - 29      770.0   94.0   864.0  0.108796
         30 - 39     1686.0  190.0  1876.0  0.101279
         40 - 49     1249.0  109.0  1358.0  0.080265
         50 - 59      451.0   37.0   488.0  0.075820
         60 - 69      143.0    6.0   149.0  0.040268
         mayor a 70     9.0    1.0    10.0  0.100000
2        19 - 29      295.0   40.0   335.0  0.119403
         30 - 39     1046.0  111.0  1157.0  0.095938
         40 - 49      463.0   41.0   504.0  0.081349
         50 - 59       76.0    6.0    82.0  0.073171
         60 - 69       11.0    NaN     NaN       NaN
3        19 - 29       33.0    3.0    36.0  0.083333
         30 - 39      176.0   17.0   193.0  0.088083
         40 - 49       78.0    7.0    85.0  0.082353
         50 - 59       10.0    NaN     NaN       NaN
         60 - 69        1.0    NaN     NaN       NaN
4        19 - 29        3.0    1.0     4.0  0.250000
         30 - 39       25.0    1.0    26.0  0.038462
         40 - 49        7.0    1.0     8.0  0.125000
         50 - 59        NaN    1.0     NaN       NaN
5        30 - 39        7.0    NaN     NaN       NaN
         40 - 49        1.0    NaN     NaN       NaN
         50 - 59        1.0    NaN     NaN       NaN

**Conclusión**

Debido a la poca diferencia de porcentaje no se puede establecer alguna correlación entre la cantidad de hijos y pagar a tiempo; sin embargo, al hacer una segunda tabla incluyendo la categorización de edades se aprecia que mientras menos edad se tenga, hay más probabilidad de deuda.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [90]:
tabla_2=clasificacion('family_status')['categ_edad']
tabla_2['total']=tabla_2[0]+tabla_2[1]
tabla_2['tasa']=tabla_2[1]/tabla_2['total']
tabla_2

debt,0,1,total,tasa
family_status,,,,
civil partnership,3763,388,4151,0.093471
married,11408,931,12339,0.075452
single,3646,359,4005,0.089638
widow / widower,896,63,959,0.065693


**Conclusión**

La situación familiar no parece influir en el pago a tiempo, salvo en el caso de las personas viudas que tienen una ligera disminución en la tasa por lo que serían menos probables de endeudarse.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [91]:
tabla_3=clasificacion('cuartil_total_income')['categ_edad']
tabla_3['total']=tabla_3[0]+tabla_3[1]
tabla_3['tasa']=tabla_3[1]/tabla_3['total']
tabla_3

debt,0,1,total,tasa
cuartil_total_income,,,,
1er cuartil,4772,592,5364,0.110365
2do cuartil,4887,476,5363,0.088756
3er cuartil,5209,356,5565,0.063971
4to cuartil,4845,317,5162,0.061410


**Conclusión**

Las personas que se encuentran en el primer cuartil de ganancias totales presentan 11.08 % de poseer deudas. A medida que aumentan las ganancias totales, disminuye la tasa de deudas.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [92]:
tabla_4=clasificacion('categ_purpose')['categ_edad']
tabla_4['total']=tabla_4[0]+tabla_4[1]
tabla_4['tasa']=tabla_4[1]/tabla_4['total']
tabla_4

debt,0,1,total,tasa
categ_purpose,,,,
business,3008,256,3264,0.078431
car,3451,361,3812,0.094701
education,3643,370,4013,0.092200
housing,7021,526,7547,0.069697
wedding,2138,186,2324,0.080034


**Conclusión**

Las tasas presentan valores similares en cuanto a la categoría del propósito. Hay un ligero incremento de riesgo de deuda si el propósito es con fines educativos o relacionados a la adquisición o renovación de un auto, mientras que hay un menor riesgo si el propósito es para fines de vivienda.

# Conclusión general 

1. No hay una relación entre tener hijos y pagar un préstamo a tiempo.
2. No hay una relación entre el estado civil y pagar un préstamo a tiempo; sin embargo, las personas viudas tienen menor capacidad de endeudarse que el resto.
3. Existe una relación directa entre el nivel de ingresos y el pago de préstamos a tiempo. Mientras el nivel de ingresos aumenta, hay menor probabilidad de endeudamiento.
4. Existe un mayor riesgo de deuda si el préstamo es para autos o fines educativos. Hay menor riesgo de deuda si es con fines de vivienda.
5. Se eliminaron 71 filas con información duplicada que equivale al 0.32 % del total.
6. Se reemplazó el 10.09 % de valores ausentes en las columnas 'days_employed' y 'total_income', los cuales fueron reemplazados por las medias y medianas, respecticamente de dichas columnas en función de 'income_type'.
